In [1]:
%reload_ext autoreload
%autoreload 2
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2' # INFO, WARNING messages are not printed
import tensorflow as tf
import time # for throughput measurements

# Configure the notebook to use only a single GPU and allocate only as much memory as needed
# For more details, see https://www.tensorflow.org/guide/gpu
gpus = tf.config.list_physical_devices('GPU')
print('Number of GPUs available :', len(gpus))
if gpus:
    gpu_num = 0 # Number of the GPU to be used
    try:
        #tf.config.set_visible_devices([], 'GPU')
        tf.config.set_visible_devices(gpus[gpu_num], 'GPU')
        print('Only GPU number', gpu_num, 'used.')
        tf.config.experimental.set_memory_growth(gpus[gpu_num], True)
    except RuntimeError as e:
        print(e)

# uninstall sionna first
import sys
from pathlib import Path

sys.path.append(str(Path('..')))

from sionna.fec.ldpc import *
from sionna.utils import BinarySource 
from sionna.utils.metrics import count_block_errors
from sionna.channel import Pauli, BinarySymmetricChannel
from sionna.utils.plotting import PlotBER

from tensorflow.keras.losses import BinaryCrossentropy
from sionna.utils.metrics import compute_bler
from sionna.fec.utils import int_mod_2, row_echelon

import time

Number of GPUs available : 1
Only GPU number 0 used.


In [2]:
# a = tf.constant([[0.7,0.2,0.3],[0.4,0.7,0.1]])
a = tf.constant([0.7,0.2,0.3,1.9,0.8,0.2])
sort_order = tf.argsort(a)
inv_sort = tf.math.invert_permutation(sort_order)
inv_sort_2 = tf.argsort(sort_order)
tf.print(sort_order)
tf.print(inv_sort)
tf.print(inv_sort_2)

[1 5 2 0 4 3]
[3 0 2 5 4 1]
[3 0 2 5 4 1]


In [3]:
t = tf.constant([[[1, 10, 100], [2, 20, 200]],
                 [[3, 30, 300], [4, 40, 400]],
                 [[5, 50, 500], [6, 60, 600]]])
print(tf.shape(t))
indices = tf.constant([[0,1], [1,2], [2,0]])
print(tf.shape(indices))
tf.gather(t, indices, batch_dims=1, axis=-1)

tf.Tensor([3 2 3], shape=(3,), dtype=int32)
tf.Tensor([3 2], shape=(2,), dtype=int32)


<tf.Tensor: shape=(3, 2, 2), dtype=int32, numpy=
array([[[  1,  10],
        [  2,  20]],

       [[ 30, 300],
        [ 40, 400]],

       [[500,   5],
        [600,   6]]], dtype=int32)>

In [36]:
indices = [[0],[1]]
print(tf.shape(indices))

tf.Tensor([2 1], shape=(2,), dtype=int32)


In [40]:
idx_pivot = [[1,3,2],[3,2,1]]
ii, _ = tf.meshgrid(tf.range(2), tf.range(3), indexing='ij')
print(ii)
idx_updates = tf.stack([ii, idx_pivot], axis=-1)
print(idx_updates)

tf.Tensor(
[[0 0 0]
 [1 1 1]], shape=(2, 3), dtype=int32)
tf.Tensor(
[[[0 1]
  [0 3]
  [0 2]]

 [[1 3]
  [1 2]
  [1 1]]], shape=(2, 3, 2), dtype=int32)


In [93]:
tensor = tf.constant([[0, 0, 0, 0, 0, 0, 0, 0], [0,0,0,0,0,0,0,0]])    # tf.rank(tensor) == 1
# indices = [[1], [7], [3], [4]]       # num_updates == 4, index_depth == 1
indices = tf.constant([[[0,1],[0,7],[0,3],[0,4]],[[1,2],[1,6],[1,3],[1,5]]])
# updates = [9, 10, 11, 12] 
updates = tf.constant([[9,10,11,12],[2,3,4,5]])
print(tensor.shape)
print(indices.shape)
print(updates.shape)
print(tf.tensor_scatter_nd_update(tensor, indices, updates))

(2, 8)
(2, 4, 2)
(2, 4)
tf.Tensor(
[[ 0  9  0 11 12  0  0 10]
 [ 0  0  2  4  0  5  3  0]], shape=(2, 8), dtype=int32)


In [2]:
class BP4_Error_Model(tf.keras.Model):
# For storing error strings that the BP decoder failed to decode
    def __init__(self, code, decoder, num_iter=32, trainable=False, loss_type="boxplus-phi", wt=False):

        super().__init__()
        
        self.k = code.K      
        self.n = code.N
        self.hx = code.hx
        self.hz = code.hz
        self.lx = code.lx
        self.lz = code.lz
        self.hx_perp = code.hx_perp # contains Im(hz.T)
        self.hz_perp = code.hz_perp # contains Im(hx.T)
        self.code_name = code.name
        self.num_checks = code.hx.shape[0] + code.hz.shape[0]
        
        self.source = BinarySource()
        self.channel = Pauli(wt=wt)
        self.decoder = decoder

        self.num_iter = num_iter
        self.trainable = trainable
        self.bce = BinaryCrossentropy(from_logits=True)
        self.loss_type = loss_type
        self.wt = wt
           
    @tf.function(jit_compile=True, reduce_retracing=True) # XLA mode
    def call(self, batch_size, ebno_db):

        p = ebno_db

        # depolarizing noise
        px, py, pz = 2*p/3, p/3, 2*p/3
        c_dummy = tf.zeros([batch_size, self.n])
        if self.wt: # ebno_db is an integer indicating the weight of the error
            noise_x, noise_z = self.channel([c_dummy, None, p])
        else:
            noise_x, noise_z = self.channel([c_dummy, None, px, py, pz])  # [bs, self.n]
        noise_x_T, noise_z_T = tf.transpose(noise_x, (1,0)), tf.transpose(noise_z, (1,0))
        noise_x_int = tf.cast(noise_x_T, self.hz.dtype)
        noise_z_int = tf.cast(noise_z_T, self.hx.dtype)
        syndrome_x = int_mod_2(tf.matmul(self.hx, noise_z_int))
        syndrome_z = int_mod_2(tf.matmul(self.hz, noise_x_int))

        llr_ch_x = tf.fill(tf.shape(noise_x), tf.math.log(3.*(1.-p)/p))
        llr = tf.tile(tf.expand_dims(llr_ch_x, axis=1), multiples=tf.constant([1, 3, 1], tf.int32))
        # shape of llr: [bs, 3, self.n]
        
        x_hat, z_hat = self.decoder((llr, syndrome_x, syndrome_z))
        
        x_hat = tf.transpose(tf.cast(x_hat, tf.bool), (1,0)) # [self.n, bs]
        z_hat = tf.transpose(tf.cast(z_hat, tf.bool), (1,0))

        x_diff = tf.cast(tf.math.logical_xor(noise_x_T, x_hat), self.hx_perp.dtype)
        z_diff = tf.cast(tf.math.logical_xor(noise_z_T, z_hat), self.hz_perp.dtype)

        sx = int_mod_2(tf.matmul(self.hz, x_diff))
        sz = int_mod_2(tf.matmul(self.hx, z_diff))
        s_hat = tf.concat([sx, sz], axis=0)
        s_hat = tf.transpose(s_hat, (1,0))
        err = tf.reduce_any(tf.not_equal(tf.zeros_like(s_hat), s_hat), axis=-1)
        return noise_x, noise_z, x_hat, z_hat, err


In [3]:
class OSD_Model(tf.keras.Model):
# For storing error strings that the BP decoder failed to decode
    def __init__(self, code, decoder, num_iter=32, trainable=False, loss_type="boxplus-phi", wt=False):

        super().__init__()
        
        self.k = code.K      
        self.n = code.N
        self.hx = code.hx
        self.hz = code.hz
        self.rank_hx, self.pivot_hx, self.hx_basis = code.rank_hx, code.pivot_hx, code.hx_basis
        self.rank_hz, self.pivot_hz, self.hz_basis = code.rank_hz, code.pivot_hz, code.hz_basis
        self.lx = code.lx
        self.lz = code.lz
        self.hx_perp = code.hx_perp # contains Im(hz.T)
        self.hz_perp = code.hz_perp # contains Im(hx.T)
        self.code_name = code.name
        self.num_checks = code.hx.shape[0] + code.hz.shape[0]
        
        self.source = BinarySource()
        self.channel = Pauli(wt=wt)
        self.decoder = decoder

        self.num_iter = num_iter
        self.trainable = trainable
        self.bce = BinaryCrossentropy(from_logits=True)
        self.loss_type = loss_type
        self.wt = wt
        
        
    @tf.function(jit_compile=True, reduce_retracing=True)
    def find_mrb(self, pcm, bs):
        # the parity check matrix must be full-rank
        # find most reliable basis for the parity check matrix of size [bs, rank, n]
        # column-wise gaussian elimination
        # find pivot for each row

        _, rank, _ = pcm.shape
        s = pcm.shape

        idx_pivot = tf.TensorArray(tf.int32, rank, dynamic_size=False)
        
        for row in tf.range(rank):
            pcm = tf.ensure_shape(pcm, s)
            
            # find pivot and store it in TensorArray
            idx_p = tf.argmax(pcm[:, row, :], axis=-1, output_type=tf.int32) # size [bs]
            idx_pivot = idx_pivot.write(row, idx_p)
            
            # gather the idx_pivot'th column from pcm
            c = tf.gather(pcm, idx_p, batch_dims=1, axis=-1) # [bs, rank]
            
            # do not eliminate current row, i.e., c[:,row]=0
            all_zero = tf.zeros((bs, 1), dtype=tf.int32)
            c = tf.concat([c[:,:row], all_zero, c[:,row+1:]], axis=1) # or use tensor_scatter_nd_update?
            c = tf.tile(tf.expand_dims(c, axis=-1), (1, 1, self.n+1)) # [bs, rank, n+1]
            
            # use current row to eliminate all other rows
            current_row = tf.expand_dims(pcm[:,row,:], axis=1) # [bs, 1 ,n]                        
            pcm = int_mod_2(pcm + c * current_row)
           
        idx_pivot = tf.transpose(idx_pivot.stack()) # [bs, rank]
        sol = tf.cast(pcm[:,:,-1], tf.bool) # last column, [bs, rank]
        return idx_pivot, sol
   
    @tf.function(jit_compile=True, reduce_retracing=True)    
    def osd0(self, llr, pcm, s, bs):
        # use llrz together with code.hx and syndrome_x, return noise_z_hat

        sort_order = tf.argsort(llr) # [bs, n]
        pcm = tf.cast(pcm, tf.int32)

        permuted_pcm = tf.gather(pcm, sort_order, batch_dims=1, axis=-1)

        inv_sort = tf.argsort(sort_order) # is it correct ???
        
        syndrome = tf.cast(tf.expand_dims(tf.transpose(s, (1,0)), -1), permuted_pcm.dtype) # [bs, rank, 1]
        pcm_syndrome = tf.concat((permuted_pcm, syndrome), axis=-1) # [bs, rank, n+1]

        idx_pivot, sol = self.find_mrb(pcm_syndrome, bs)


        # [bs, rank, rank] * [bs, rank, 1] = [bs, rank, 1]
        _, rank = sol.shape
        ii, _ = tf.meshgrid(tf.range(bs), tf.range(rank), indexing='ij')
        # for sample i in the batch, update using idx_pivot[i]
        
        ii = tf.cast(ii, tf.int32)
        idx_pivot = tf.cast(idx_pivot, tf.int32)
        idx_updates = tf.stack([ii, idx_pivot], axis=-1) # [bs, rank, 2]

        e_hat = tf.tensor_scatter_nd_update(tf.zeros_like(llr, dtype=tf.bool), idx_updates, sol)
        e_hat = tf.gather(e_hat, inv_sort, batch_dims=1, axis=-1)

        return e_hat        

    
    @tf.function(jit_compile=True, reduce_retracing=True) # XLA mode
    def call(self, noise_x, noise_z, p, batch_size):    
#         batch_size = noise_x.shape[0]
        noise_x_T, noise_z_T = tf.transpose(noise_x, (1,0)), tf.transpose(noise_z, (1,0))
        noise_x_int = tf.cast(noise_x_T, self.hz.dtype)
        noise_z_int = tf.cast(noise_z_T, self.hx.dtype)
        syndrome_x = int_mod_2(tf.matmul(self.hx, noise_z_int))
        syndrome_z = int_mod_2(tf.matmul(self.hz, noise_x_int))

        llr_ch_x = tf.fill(tf.shape(noise_x), tf.math.log(3.*(1.-p)/p))
        llr = tf.tile(tf.expand_dims(llr_ch_x, axis=1), multiples=tf.constant([1, 3, 1], tf.int32))
        # shape of llr: [bs, 3, self.n]
        
        llrx, llry, llrz, x_hat, z_hat, _, _ = self.decoder((llr, syndrome_x, syndrome_z))
       
        x_hat = tf.transpose(tf.cast(x_hat, tf.bool), (1,0)) # [self.n, bs]
        z_hat = tf.transpose(tf.cast(z_hat, tf.bool), (1,0))

        x_diff = tf.cast(tf.math.logical_xor(noise_x_T, x_hat), self.hx_perp.dtype)
        z_diff = tf.cast(tf.math.logical_xor(noise_z_T, z_hat), self.hz_perp.dtype)

        sx = int_mod_2(tf.matmul(self.hz, x_diff))
        sz = int_mod_2(tf.matmul(self.hx, z_diff))
        s_hat = tf.concat([sx, sz], axis=0)
        s_hat = tf.transpose(s_hat, (1,0))
        err = tf.reduce_any(tf.not_equal(tf.zeros_like(s_hat), s_hat), axis=-1)
         
        reduced_sx = tf.gather(syndrome_x, self.pivot_hx, axis=0) # [rank, new_bs]
        reduced_sz = tf.gather(syndrome_z, self.pivot_hz, axis=0) # [rank, new_bs]
        
        num_hx = tf.math.softplus(-1.*llrx) # I or X, both commute with X-type check
        denom_hx = tf.ragged.map_flat_values(lambda x, y: tf.math.reduce_logsumexp(-1.*tf.stack([x, y], axis=-1), axis=-1), llrz, llry)
        new_llr_z = num_hx - denom_hx

        num_hz = tf.math.softplus(-1.*llrz) # I or Z, both commute with Z-type check
        denom_hz = tf.ragged.map_flat_values(lambda x, y: tf.math.reduce_logsumexp(-1.*tf.stack([x, y], axis=-1), axis=-1), llrx, llry)
        new_llr_x = num_hz - denom_hz
        

        full_rank_hx = tf.broadcast_to(tf.expand_dims(self.hx_basis, axis=0),
                             (batch_size, self.rank_hx, self.n))
        full_rank_hz = tf.broadcast_to(tf.expand_dims(self.hz_basis, axis=0),
                             (batch_size, self.rank_hz, self.n))

        z_hat_osd = self.osd0(new_llr_z, full_rank_hx, reduced_sx, batch_size)
        x_hat_osd = self.osd0(new_llr_x, full_rank_hz, reduced_sz, batch_size)
        
        return x_hat_osd, z_hat_osd
        

In [7]:
class BP4_OSD_Model(tf.keras.Model):

    def __init__(self, code, bp4_model, osd_model):

        super().__init__()
        self.bp4_model = bp4_model
        self.osd_mode = osd_model

        self.hx_perp = code.hx_perp # contains Im(hz.T)
        self.hz_perp = code.hz_perp # contains Im(hx.T)
        
    def call(self, batch_size, ebno_db):
        noise_x, noise_z, x_hat, z_hat, err = self.bp4_model(batch_size, ebno_db)
        nx, nz = noise_x[err], noise_z[err]
        new_bs = tf.reduce_sum(tf.cast(err, tf.int32))
        x_hat_osd, z_hat_osd = osd_model(nx, nz, ebno_db, new_bs)
        
        new_x_hat = tf.tensor_scatter_nd_update(tf.transpose(x_hat, (1,0)), tf.where(err), x_hat_osd)
        new_z_hat = tf.tensor_scatter_nd_update(tf.transpose(z_hat, (1,0)), tf.where(err), z_hat_osd)
        
        new_x_hat_T = tf.transpose(new_x_hat, (1,0))
        new_z_hat_T = tf.transpose(new_z_hat, (1,0))
        x_diff = tf.cast(tf.math.logical_xor(tf.transpose(noise_x, (1,0)), new_x_hat_T), self.hx_perp.dtype)
        z_diff = tf.cast(tf.math.logical_xor(tf.transpose(noise_z, (1,0)), new_z_hat_T), self.hz_perp.dtype)
  
        lsx = int_mod_2(tf.matmul(self.hx_perp, x_diff))
        lsz = int_mod_2(tf.matmul(self.hz_perp, z_diff))
            
        ls_hat = tf.concat([lsx, lsz], axis=0)      # for total logical error counts

        ls_hat = tf.transpose(ls_hat, (1,0))         # bs should be the first dimension!!!

        return tf.zeros_like(ls_hat), ls_hat     


In [8]:
start_time = time.perf_counter()

GHP_n882_k24 = create_QC_GHP_codes(63, create_cyclic_permuting_matrix(7, [27,54,0]), [0,1,6]) # 18 <= d <= 24
code = GHP_n882_k24
# GHP_n1270_k28 = create_QC_GHP_codes(127, np.array([[0,-1,51,52,-1],[-1,0,-1,111,20],[0,-1,98,-1,122],[0,80,-1,119,-1],[-1,0,5,-1,106]]), [0,1,7], name="GHP_n1270_k28") # 16 <= d <= 46
# code = GHP_n1270_k28
num_iter = tf.constant(100)
factor=tf.constant(0.8)

decoder_hard = QLDPCBPDecoder(code=code, num_iter=num_iter, normalization_factor=factor, cn_type="minsum", trainable=False, stage_one=False)
decoder_soft = QLDPCBPDecoder(code=code, num_iter=num_iter, normalization_factor=factor, cn_type="minsum", trainable=False, stage_one=True)

bp4_model = BP4_Error_Model(code, decoder_hard, num_iter=num_iter, trainable=False, wt=False)
osd_model = OSD_Model(code, decoder_soft, num_iter=num_iter, trainable=False, wt=False)
bp4_osd_model = BP4_OSD_Model(code, bp4_model, osd_model)
batch_size = tf.constant(50000)
p = tf.constant(0.09)

# for i in range(10):
#     start_time = time.perf_counter()
#     bp4_osd_model(batch_size, p)
#     end_time = time.perf_counter()
#     print("Elapsed time: ", end_time-start_time)



In [9]:
p_range = np.arange(0.08,0.101,0.01)[::-1]

ber_plot = PlotBER("Performance of the [[882,24]] code on BSC channel under binary decoding")

ber_plot.simulate(bp4_osd_model, 
                  ebno_dbs=p_range, # physical error rates to simulate
                  legend=f"{code.name}, factor={factor}, iter={num_iter}", # legend string for plotting
                  max_mc_iter=1000, # run 1000 Monte Carlo runs per physical error rate point
                  num_target_block_errors=100, # continue with next physical error rate point after 1000 block errors
                  batch_size=50000, # batch-size per Monte Carlo run
                  soft_estimates=False, # the model returns hard-estimates
                  early_stop=True, # stop simulation if no error has been detected at current physical error rate
                  show_fig=False, # do not show the figure after all results are simulated
                  add_bler=True, # we are interested in block error rate
                  qldpc=False, # since there is no flagged error for bp-osd
                  forward_keyboard_interrupt=True, # should be True in a loop
                  graph_mode=None);

EbNo [dB] |        BER |       BLER |  bit errors |    num bits | block errors |  num blocks | runtime [s] |    status
---------------------------------------------------------------------------------------------------------------------------------------
      0.1 | 5.9308e-05 | 3.4667e-04 |       16120 |   271800000 |          104 |      300000 |        95.6 |reached target block errors
     0.09 | 8.6908e-06 | 5.1795e-05 |       15354 |  1766700000 |          101 |     1950000 |       454.6 |reached target block errors
     0.08 | 1.1202e-06 | 7.0922e-06 |       14310 | 12774600000 |          100 |    14100000 |      2440.1 |reached target block errors


In [7]:
tf.print(tf.shape(noise_x))
noise_x_T, noise_z_T = tf.transpose(noise_x, (1,0)), tf.transpose(noise_z, (1,0))
noise_x_int = tf.cast(noise_x_T, code.hz.dtype)
noise_z_int = tf.cast(noise_z_T, code.hx.dtype)
syndrome_x = int_mod_2(tf.matmul(code.hx, noise_z_int))
syndrome_z = int_mod_2(tf.matmul(code.hz, noise_x_int))
tf.print(tf.shape(syndrome_x))
tf.print(syndrome_x)
tf.print(syndrome_z)

llr_ch_x = tf.fill(tf.shape(noise_x), tf.math.log(3.*(1.-p)/p))
llr = tf.tile(tf.expand_dims(llr_ch_x, axis=1), multiples=tf.constant([1, 3, 1], tf.int32))

llrx, llry, llrz, x_hat_stage_one, z_hat_stage_one, logit_hx_perp, logit_hz_perp = decoder_soft((llr, syndrome_x, syndrome_z))
tf.print("llrx shape", tf.shape(llrx))
tf.print(llrx)


[119 882]
[441 119]
[[0 1 1 ... 0 0 0]
 [0 1 0 ... 0 0 0]
 [0 0 1 ... 0 0 1]
 ...
 [0 1 0 ... 0 0 1]
 [0 1 0 ... 1 0 0]
 [0 0 0 ... 0 1 0]]
[[0 0 0 ... 0 0 1]
 [1 1 1 ... 0 0 0]
 [1 0 1 ... 1 1 0]
 ...
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 1]
 [0 1 0 ... 0 1 1]]
llrx shape [119 882]
[[43.3375244 43.3375244 -36.513031 ... -36.513031 43.3375244 43.3375244]
 [43.3375244 43.3375244 43.3375244 ... 43.3375244 43.3375244 43.3375244]
 [2.68565249 26.255352 -24.9922771 ... 22.6572895 22.4827442 23.6940384]
 ...
 [43.3375244 43.3375244 43.3375244 ... 43.3375244 43.3375244 43.3375244]
 [43.3375244 43.3375244 43.3375244 ... 43.3375244 43.3375244 43.3375244]
 [-2.61972761 23.6043301 41.1194534 ... 43.3375244 43.3375244 -33.7404442]]


In [36]:
num_hx = tf.math.softplus(-1.*llrx) # I or X, both commute with X-type check
denom_hx = tf.ragged.map_flat_values(lambda x, y: tf.math.reduce_logsumexp(-1.*tf.stack([x, y], axis=-1), axis=-1), llrz, llry)
new_llr_z = num_hx - denom_hx
tf.print(tf.shape(new_llr_z))

num_hz = tf.math.softplus(-1.*llrz) # I or Z, both commute with Z-type check
denom_hz = tf.ragged.map_flat_values(lambda x, y: tf.math.reduce_logsumexp(-1.*tf.stack([x, y], axis=-1), axis=-1), llrx, llry)
new_llr_x = num_hz - denom_hz
tf.print(tf.shape(new_llr_x))

[116 882]
[116 882]


In [37]:
def osd0(llr, pcm, s):
    # use llrz together with code.hx and syndrome_x, return noise_z_hat
    sort_order = tf.argsort(llr)

    permuted_pcm = tf.gather(pcm, sort_order, axis=1) 

    inv_sort = tf.math.invert_permutation(sort_order)
    
    row_ech_form, rank, transform, pivot_cols = row_echelon(permuted_pcm.numpy())
    
    new_pcm = tf.gather(permuted_pcm, pivot_cols, axis=1)
    
    syndrome = tf.expand_dims(s, 1)
    
    pcm_synd = tf.concat((new_pcm, syndrome), axis=1)
    
    row_ech_form, rank, _, _ = row_echelon(pcm_synd.numpy(), reduced=True)
    
#     tf.print(tf.shape(pivot_cols))
#     tf.print(tf.shape(row_ech_form[:,-1]))
    
    e_hat = tf.tensor_scatter_nd_update(tf.zeros_like(llr, dtype=tf.bool), tf.expand_dims(pivot_cols, axis=1), row_ech_form[:,-1])
    
    e_hat = tf.gather(e_hat, inv_sort)
    
    return e_hat

In [38]:
_, _, _, pivot_hx = row_echelon(code.hx.T)
reduced_hx = tf.gather(code.hx, pivot_hx, axis=0) # select linearly independent rows
tf.print(tf.shape(reduced_hx))

_, _, _, pivot_hz = row_echelon(code.hz.T)
reduced_hz = tf.gather(code.hz, pivot_hz, axis=0) # select linearly independent rows
tf.print(tf.shape(reduced_hz))

reduced_sx = tf.gather(syndrome_x, pivot_hx, axis=0)
reduced_sz = tf.gather(syndrome_z, pivot_hz, axis=0)

tf.print(tf.shape(reduced_sx))
tf.print(tf.shape(reduced_sz))

[429 882]
[429 882]
[429 116]
[429 116]


In [39]:
noise_z_hat_list, noise_x_hat_list = [], []
bs = tf.shape(new_llr_z)[0]
for i in range(bs):
    noise_z_hat_list.append(osd0(new_llr_z[i], reduced_hx, reduced_sx[:,i]))
    noise_x_hat_list.append(osd0(new_llr_x[i], reduced_hz, reduced_sz[:,i]))
    
noise_x_hat = tf.stack(noise_x_hat_list, axis=0)
tf.print(tf.shape(noise_x_hat))
noise_z_hat = tf.stack(noise_z_hat_list, axis=0)
tf.print(tf.shape(noise_z_hat))

[116 882]
[116 882]


In [40]:
x_hat = tf.transpose(noise_x_hat, (1,0)) # [self.n, bs]
z_hat = tf.transpose(noise_z_hat, (1,0))
noise_x_T, noise_z_T = tf.transpose(noise_x, (1,0)), tf.transpose(noise_z, (1,0))
print(tf.shape(noise_x_T))
print(tf.shape(x_hat))

x_diff = tf.cast(tf.math.logical_xor(noise_x_T, x_hat), code.hx_perp.dtype)
z_diff = tf.cast(tf.math.logical_xor(noise_z_T, z_hat), code.hz_perp.dtype)

sx = int_mod_2(tf.matmul(code.hz, x_diff))
sz = int_mod_2(tf.matmul(code.hx, z_diff)) 
s_hat = tf.concat([sx, sz], axis=0)
s_hat = tf.transpose(s_hat, (1,0))
err = tf.reduce_any(tf.not_equal(tf.zeros_like(s_hat), s_hat), axis=-1)
# OSD should clean up all flag errors

lsx = int_mod_2(tf.matmul(code.hx_perp, x_diff))
lsz = int_mod_2(tf.matmul(code.hz_perp, z_diff))

ls_hat = tf.concat([lsx, lsz], axis=0)      # for total logical error counts
ls_hat = tf.transpose(ls_hat, (1,0))         # bs should be the first dimension!!!
logical_err = tf.reduce_any(tf.not_equal(tf.zeros_like(ls_hat), ls_hat), axis=-1)

print(tf.shape(s_hat))
print(tf.reduce_sum(s_hat))
print(tf.reduce_sum(tf.cast(logical_err, dtype=tf.int32)))

tf.Tensor([882 116], shape=(2,), dtype=int32)
tf.Tensor([882 116], shape=(2,), dtype=int32)
tf.Tensor([116 882], shape=(2,), dtype=int32)
tf.Tensor(40690, shape=(), dtype=int64)
tf.Tensor(116, shape=(), dtype=int32)


## Quaternary BP
| CN update       | $p$  | $p_L$  |
| --------------- | :-:  | :-:    |
| SP, 1.0, 64     | 0.10 | 5.1e-3
|                 | 0.09 | 1.74e-3
|                 | 0.08 | 1.08e-3
| NMS, 0.8, 100   | 0.10 | 2.8e-4
|                 | 0.09 | 2e-5 (1/50000)


In [17]:
code = GHP_n882_k24
decoder = LDPCBPDecoder(code.hx, is_syndrome=True, num_iter=64, cn_type='minsum') # binary syndrome BP decoder
# noise added by BSC channel
model = BP_BSC_Model(pcm=code.hx, decoder=decoder, logical_pcm=code.hz_perp, p0=0.2) # p0 is used to initialize BP
# If p0 is not specified (default set to None), then the actual p value employed for noise generation will be used for BP initialization.
p_range = np.arange(0.01,0.101,0.01)[::-1] * 2/3 

ber_plot = PlotBER("Performance of the [[882,24]] code on BSC channel under binary decoding")

ber_plot.simulate(model, 
                  ebno_dbs=p_range, # physical error rates to simulate
                  legend=f"{code.name}, factor={1.0}, iter={64}, p0=0.2", # legend string for plotting
                  max_mc_iter=1000, # run 1000 Monte Carlo runs per physical error rate point
                  num_target_block_errors=100, # continue with next physical error rate point after 1000 block errors
                  batch_size=10000, # batch-size per Monte Carlo run
                  soft_estimates=False, # the model returns hard-estimates
                  early_stop=True, # stop simulation if no error has been detected at current physical error rate
                  show_fig=False, # do not show the figure after all results are simulated
                  add_bler=True, # we are interested in block error rate
                  qldpc=True, # show flagged error rate, instead of showing bit error rate
                  forward_keyboard_interrupt=True); # should be True in a loop

        p |    Flagged |       BLER | flag errors | block errors |  num blocks | runtime [s] |    status
---------------------------------------------------------------------------------------------------------------------------------------
    0.067 | 1.0000e+00 | 1.0000e+00 |       10000 |        10000 |       10000 |         1.2 |reached target block errors
     0.06 | 9.9730e-01 | 9.9730e-01 |        9973 |         9973 |       10000 |         0.2 |reached target block errors
    0.053 | 9.8540e-01 | 9.8540e-01 |        9854 |         9854 |       10000 |         0.2 |reached target block errors
    0.047 | 9.1680e-01 | 9.1680e-01 |        9168 |         9168 |       10000 |         0.2 |reached target block errors
     0.04 | 7.0030e-01 | 7.0030e-01 |        7003 |         7003 |       10000 |         0.2 |reached target block errors
    0.033 | 3.6040e-01 | 3.6040e-01 |        3604 |         3604 |       10000 |         0.2 |reached target block errors
    0.027 | 1.0610e-01 | 1.

In [ ]:
class BP2_BSC_Error_Model(tf.keras.Model):
    def __init__(self, pcm, decoder, logical_pcm=None, p0=None):

        super().__init__()
        
        # store values internally
        self.pcm = pcm
        self.logical_pcm = logical_pcm
        _, self.n = pcm.shape
        self.source = BinarySource()
       
        self.channel = BinarySymmetricChannel()

        # FEC encoder / decoder
        self.decoder = decoder
        self.p0 = p0

#     @tf.function(jit_compile=True, reduce_retracing=True) # XLA mode during evaluation
    @tf.function() # graph mode
    def call(self, batch_size, ebno_db):

        p0 = ebno_db if self.p0 is None else self.p0
        llr_const = -tf.math.log((1.-p0)/p0)
           
        c = tf.zeros([batch_size, self.n])
        noise = self.channel((c, ebno_db))  # [bs, self.n]
        llr = tf.fill(tf.shape(noise), llr_const)  
        noise_int = tf.transpose(tf.cast(noise, self.pcm.dtype), (1,0)) # [self.n, bs]
        syndrome = int_mod_2(tf.matmul(self.pcm, noise_int))
        noise_hat = self.decoder((llr, syndrome)) 

        noise_hat = tf.transpose(tf.cast(noise_hat, self.pcm.dtype), (1,0)) # [self.n, bs]
        noise_diff = tf.math.logical_xor(tf.cast(noise_int, tf.bool), tf.cast(noise_hat, tf.bool)) 
        noise_diff = tf.cast(noise_diff, self.pcm.dtype)
        s_hat = int_mod_2(tf.matmul(self.pcm, noise_diff))
        s_hat = tf.transpose(s_hat, (1,0)) # bs should be the first dimension!

        err = tf.reduce_any(tf.not_equal(tf.zeros_like(s_hat), s_hat), axis=-1)
        return noise[err]

In [65]:
GHP_n882_k24 = create_QC_GHP_codes(63, create_cyclic_permuting_matrix(7, [27,54,0]), [0,1,6]) # 18 <= d <= 24
code = GHP_n882_k24

p = tf.constant(0.04)
num_iter = 100
factor = tf.constant(0.8)

decoder_hard = LDPCBPDecoder(code.hx, cn_type="minsum", is_syndrome=True, num_iter=num_iter, normalization_factor=factor, hard_out=True) # binary syndrome BP decoder
decoder_soft = LDPCBPDecoder(code.hx, cn_type="minsum", is_syndrome=True, num_iter=num_iter, normalization_factor=factor, hard_out=False) # binary syndrome BP decoder

model = BP2_BSC_Error_Model(pcm=code.hx, decoder=decoder_hard, p0=p)

batch_size = tf.constant(50000)

print(f"physical error rate {p}, find BP failed-to-decode samples (flagged errors)")

sample_list = []

noise = model(batch_size, p)
sample_list.append(noise)


physical error rate 0.03999999910593033, find BP failed-to-decode samples (flagged errors)


In [66]:
noise = sample_list[0]
tf.print(tf.shape(noise))
noise_T = tf.transpose(noise, (1,0))
noise_int = tf.cast(noise_T, code.hx.dtype)
syndrome = int_mod_2(tf.matmul(code.hx, noise_int))
tf.print(tf.shape(syndrome))

llr = tf.fill(tf.shape(noise), -tf.math.log((1.-p)/p))
llr_post = decoder_soft((llr, syndrome))


[1190 882]
[441 1190]


In [67]:
_, _, _, pivot_hx = row_echelon(code.hx.T)
reduced_hx = tf.gather(code.hx, pivot_hx, axis=0) # select linearly independent rows
tf.print(tf.shape(reduced_hx))

reduced_s = tf.gather(syndrome, pivot_hx, axis=0)

tf.print(tf.shape(reduced_s))

noise_hat_list = []
bs = tf.shape(llr_post)[0]
for i in range(bs):
    noise_hat_list.append(osd0(-1.*llr_post[i], reduced_hx, reduced_s[:,i]))
    
noise_hat = tf.stack(noise_hat_list, axis=0)
tf.print(tf.shape(noise_hat))

[429 882]
[429 1190]
[1190 882]


In [68]:
noise_hat = tf.transpose(noise_hat, (1,0)) # [self.n, bs]
print(tf.shape(noise_T))
print(tf.shape(noise_hat))
noise_diff = tf.cast(tf.math.logical_xor(tf.cast(noise_T, tf.bool), noise_hat), code.hz_perp.dtype)

s_hat = int_mod_2(tf.matmul(code.hx, noise_diff)) 
s_hat = tf.transpose(s_hat, (1,0))
err = tf.reduce_any(tf.not_equal(tf.zeros_like(s_hat), s_hat), axis=-1)
# OSD should clean up all flag errors

ls_hat = int_mod_2(tf.matmul(code.hz_perp, noise_diff))

ls_hat = tf.transpose(ls_hat, (1,0))         # bs should be the first dimension!!!
logical_err = tf.reduce_any(tf.not_equal(tf.zeros_like(ls_hat), ls_hat), axis=-1)

print(tf.shape(s_hat))
print(tf.reduce_sum(s_hat))
print(tf.reduce_sum(tf.cast(logical_err, dtype=tf.int32)))

tf.Tensor([ 882 1190], shape=(2,), dtype=int32)
tf.Tensor([ 882 1190], shape=(2,), dtype=int32)
tf.Tensor([1190  441], shape=(2,), dtype=int32)
tf.Tensor(0, shape=(), dtype=int64)
tf.Tensor(1, shape=(), dtype=int32)


## Binary BP
| CN update       | $p$  | $p_L$  |
| --------------- | :-:  | :-:    |
| NMS, 0.8, 100   | 0.05 | 6.4e-4
| ^^              | 0.04 | 2e-5 (1/50000)